# How to select rows and columns

To answers your questions you typically do not need all the data that are present in the input file.

This notebook shows 
- how to select the observed cases (i.e. rows). 
- how to select the variables (i.e. columns).

We will demonstrate two ways to do that:
- the prefered way that uses SQL  
(We will use [pandasql](http://blog.yhat.com/posts/pandasql-intro.html) because it can do sql directly on dataframes and returns a dataframe)
- the alternative way that uses native pandas commands

# 0 - setup notebook

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
#-- install pandasql if necessary ----
#-- see: http://blog.yhat.com/posts/pandasql-intro.html ----
from pandasql import * 

# 1 - get some data

In [2]:
cities = pd.read_excel('./dat/city.xlsx', sheetname='citydata')
print(cities.shape)
cities.head()

(69, 42)


,city,areaC,areaM,popC,popM,fornB,growP,ppp,share,unempR,...,lfExpF,nrMDs,nrHosp,asLegi,nrMus,nrArts,greens,airQ,effLaw,retFit
0,London,321.0,1584.0,8.20,9.01,0.370,0.009,52.0,0.032,0.083,...,83.3,NaN,255.0,1.0,237.0,307.0,0.14,29.0,1.0,1.0
1,Amsterdam,165.0,807.0,0.76,1.40,0.473,0.012,46.0,0.010,0.054,...,80.8,269.0,7.0,1.0,68.0,141.0,0.57,24.0,1.0,0.0
2,Ankara,31.0,25437.0,3.54,4.77,NaN,0.257,21.2,NaN,0.121,...,NaN,NaN,7.0,1.0,36.0,NaN,0.07,46.0,0.0,0.0
3,Athens,39.0,381.0,0.66,4.01,0.220,NaN,30.5,NaN,0.162,...,NaN,NaN,23.0,1.0,47.0,NaN,NaN,41.0,1.0,NaN
4,Bangkok,1569.0,7762.0,5.72,6.50,0.200,0.031,23.4,0.002,0.022,...,74.0,NaN,173.0,NaN,27.0,NaN,0.24,54.0,1.0,1.0


The dataframe contains data for 69 of the largest cities in the world.  
The cities are described by 42 variables (see the appendix at the end of this notebook for a dictionary).  

We want to know if the number of hospitals is proportional to the number of people that live in the city.  
We are only interested in very large cities with more than 10 million inhabitants.

In the appendix we can find that
- the column city contains the city name
- the column popC contains the City Population  in millions of people
- the column nrHosp contains the Number of Hospitals


# 2 - Select with SQL

With one simple SQL query we can select all the data we need from the cities dataframe. 

In [6]:
large_cities = sqldf("""
SELECT city, popC, nrHosp
FROM cities
WHERE popC > 10.00
""")

large_cities

,city,popC,nrHosp
0,Beijing,14.39,551.0
1,Delhi,16.75,24.0
2,Karachi,14.50,18.0
3,Lagos,10.20,11.0
4,Moscow,10.56,252.0
5,Mumbai,11.98,32.0
6,Sao Paulo,11.25,101.0
7,Seoul,10.58,223.0
8,Shanghai,23.02,296.0
9,Tokyo,12.80,658.0


The SQL command is straightforward.
- **SELECT city, popC, nrHosp** tels SQL that we want to see the contents of the columns city, popC and nrHosp.
- **FROM cities** tels SQL from which dataframe the data should be fetched.
- **WHERE popC > 10.00** tels SQL that we only want data from cities where the population is larger than 10.00 million

The WHERE clause can be made quit complex, e.g.

> WHERE popC > 5.00 AND ppp < 50 AND (train=1 OR metro=1)

would select all cities that have population larger than 5 million, and a ppp smaller than 50000 $ and or a train system, or a metro system or both. 

In [7]:
complex = sqldf("""
SELECT city, popC, ppp, train, metro
FROM cities
WHERE popC > 5.00 AND ppp < 50 AND (train=1 OR metro=1)
""")

complex

,city,popC,ppp,train,metro
0,Bangkok,5.72,23.4,1,0
1,Istanbul,9.89,22.8,0,1
2,Jakarta,9.60,7.2,1,0
3,Osaka,8.80,35.2,1,0
4,Sao Paulo,11.25,23.7,0,1


# 3 - Select with pandas

Using standard pandas we could take two steps.
- first make a new dataframe that only contains the columns city, popC and nrHosp 
- from that new dataframe select the cities with a popC > 10.00.

In [13]:
selected_cols = cities[['city','popC','nrHosp']]

In [11]:
large_cities = selected_cols[cities['popC'] > 10.00]
large_cities

,city,popC,nrHosp
6,Beijing,14.39,551.0
16,Delhi,16.75,24.0
33,Karachi,14.50,18.0
35,Lagos,10.20,11.0
44,Moscow,10.56,252.0
45,Mumbai,11.98,32.0
54,Sao Paulo,11.25,101.0
55,Seoul,10.58,223.0
56,Shanghai,23.02,296.0
63,Tokyo,12.80,658.0


True pandas experts might do the same with a one-liner

In [42]:
cities[['city','popC','nrHosp']][cities['popC'] > 10.00]

,city,popC,nrHosp
6,Beijing,14.39,551.0
16,Delhi,16.75,24.0
33,Karachi,14.50,18.0
35,Lagos,10.20,11.0
44,Moscow,10.56,252.0
45,Mumbai,11.98,32.0
54,Sao Paulo,11.25,101.0
55,Seoul,10.58,223.0
56,Shanghai,23.02,296.0
63,Tokyo,12.80,658.0


Here is the one-liner for our second query

In [25]:
cities[['city','popC','ppp','train','metro']][ (cities['popC']>5.0) & (cities['ppp']<50) & ((cities['train']==1) | (cities['metro']==1))]

,city,popC,ppp,train,metro
4,Bangkok,5.72,23.4,1,0
29,Istanbul,9.89,22.8,0,1
30,Jakarta,9.60,7.2,1,0
47,Osaka,8.80,35.2,1,0
54,Sao Paulo,11.25,23.7,0,1


A more readable way to do the same would be
- create a list **selected_cols** with all the columns you want to select
- create a logical filter **selected_rows** to select the rows you need
- get the result with **dataframe[selected_cols][selected_rows]**

In [30]:
selected_cols = ['city','popC','ppp','train','metro']
selected_rows = (cities['popC']>5.0) & (cities['ppp']<50) & ((cities['train']==1) | (cities['metro']==1))
cities[selected_cols][selected_rows]

,city,popC,ppp,train,metro
4,Bangkok,5.72,23.4,1,0
29,Istanbul,9.89,22.8,0,1
30,Jakarta,9.60,7.2,1,0
47,Osaka,8.80,35.2,1,0
54,Sao Paulo,11.25,23.7,0,1


The pandas version of the second query is clearly more convoluted than the SQL version  
(note more convoluted means harder to write, check and maintain and harder to communicate).

# 6 - pandas slicing and dicing 

You should consider to use pandas when you want to select a number of adjacent rows or columns.

Suppose we only want to see the transportation systems variables.  
The relevant data are in the adjacent columns bikeshare, bus, ... tram.  
We can select these data with a compact command

In [33]:
trans_syst = cities.loc[:,'bikeshare':'tram']
trans_syst.head()

,bikeshare,bus,ferry,metro,rail,subway,train,tram
0,1,1,1,0,1,1,0,0
1,0,1,1,1,0,0,0,1
2,0,1,0,0,1,1,0,0
3,0,1,1,1,0,0,1,1
4,1,1,0,0,0,1,1,0


The **.loc[rowsx,colsx]** functions locates (i.e. selects) the rows specified by rowsx and the columns specified by colsx.  
In our case
- The : specifies all the rows.  
- The 'bikeshare':'tram' specifies the columns 'bikeshare' up to **and including** 'tram'.

Note that this only works for adjacent columns. If you wanted to add the column city to the above table you can not do  
> cities.loc[:,['city','bikeshare':'tram']]

That gives a syntax error (you can do it but only as demonstrated in section 5)

--------

### .loc also works well with adjacent rows.  
When we want to select only the rows 10,11,12,13 and 15 we could do that like this 

In [37]:
trans_syst = cities.loc[10:15,'bikeshare':'tram']
trans_syst

,bikeshare,bus,ferry,metro,rail,subway,train,tram
10,1,1,0,1,0,0,1,1
11,1,1,0,0,1,1,0,0
12,0,1,0,0,0,1,0,0
13,1,1,0,0,0,1,1,0
14,1,1,0,0,1,0,0,1
15,1,1,1,1,0,1,1,0


**NOTE** 10:15 means 10 up to **and including** 15

## .iloc[] versus .loc[]

The .loc function works with the labels that are found in the index and the column-index.  
There is an other function .iloc that locates by integer (not by label).  

For example the bikeshare:tram columns are the columns with number 11,12, ... 18.  
We can do the same selection using the .iloc function

**NOTE** we now need the ranges 
- 10:16 i.e 10 up to **but not including** 16
- 11:19 i.e 11 up to **but not including** 19

In [41]:
trans_syst = cities.iloc[10:16,11:19]
trans_syst

,bikeshare,bus,ferry,metro,rail,subway,train,tram
10,1,1,0,1,0,0,1,1
11,1,1,0,0,1,1,0,0
12,0,1,0,0,0,1,0,0
13,1,1,0,0,0,1,1,0
14,1,1,0,0,1,0,0,1
15,1,1,1,1,0,1,1,0


# 7 - Conclusion

Everything SQL can do, pandas can do (can do in several ways!)  
The pandas way of selecting quickly becomes convoluted and confusing (i.e. something [real programmers](https://xkcd.com/378/) love)

# Appendix - description of variables in cities.

In [4]:
codebook = pd.read_excel('./dat/city.xlsx', sheetname='dictionary')
codebook.head(50)

,variable name,description,variable group,domain type,domain constraint,measurement unit,missing values
0,city,English name of the city,NaN,string,identifier (unique and not null),NaN,not allowed
1,areaC,City Area (km2),Geography,real,>=0 with 0 decimals,km^2,empty cell
2,areaM,Metro Area (km2),Geography,real,>=0 with 0 decimals,km^2,empty cell
3,popC,City Population (millions),People,real,>=0 up to 2 decimals,in milions of people,empty cell
4,popM,Metro Population (millions),People,real,>=0 up to 2 decimals,in milions of people,empty cell
5,fornB,Foreign Born (fraction of population),People,real,">= 0 and <= 1, up to 3 decimals",NaN,empty cell
6,growP,Annual Population Growth (fraction),People,real,">= 0 and <= 1, up to 3 decimals",NaN,empty cell
7,ppp,PPP Per Capita (thousands $),Economy,real,">=0, up to 1 decimal",in kilo-$ of year ????,empty cell
8,share,Share of Global 500 Companies (fraction),Economy,real,">= 0 and <= 1, up to 3 decimals",NaN,empty cell
9,unempR,Unemployment Rate (fraction),Economy,real,">= 0 and <= 1, up to 3 decimals",NaN,empty cell
